In [1]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader

from tqdm.auto import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

/home/jovyan/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
MODEL_CKPT = 'roberta-base'
S_ROBERTA  = 's-nlp/roberta_toxicity_classifier'
DEVICE = 'cuda:4' if torch.cuda.is_available else 'cpu'
DEVICE2 = 'cuda:3' if torch.cuda.is_available else 'cpu'
BATCH_SIZE = 32

##### Simple filtering

In [3]:
train = pd.read_csv('../../data/paraphrase/output/train_gen_pair_sampled.txt', sep='\t')
dev = pd.read_csv('../../data/paraphrase/output/val_gen_pair.txt', sep='\t')
test = pd.read_csv('../../data/paraphrase/output/test_gen_pair.txt', sep='\t')

In [4]:
train20k = pd.read_csv('../../data/paraphrase/output/train_gen_pair_20k.txt', sep='\t')

In [14]:
train['gen'] = train.gen.fillna('')
dev['gen'] = dev.gen.fillna('')
test['gen'] = test.gen.fillna('')

In [5]:
train20k['gen'] = train20k.gen.fillna('')

In [6]:
# further cleaning
train['ori'] = train['ori'].apply(lambda x: x.replace('toxic: ', '').strip())
train['gen'] = train['gen'].apply(lambda x: x[:x.index('</s>')].replace('normal ', ''.strip()) \
                                   if '</s>' in x else x.replace('normal ', '').strip())

dev['ori'] = dev['ori'].apply(lambda x: x.replace('toxic: ', '').strip())
dev['gen'] = dev['gen'].apply(lambda x: x[:x.index('</s>')].replace('normal ', ''.strip()) \
                                   if '</s>' in x else x.replace('normal ', '').strip())

test['ori'] = test['ori'].apply(lambda x: x.replace('toxic: ', '').strip())
test['gen'] = test['gen'].apply(lambda x: x[:x.index('</s>')].replace('normal ', ''.strip()) \
                                   if '</s>' in x else x.replace('normal ', '').strip())

In [6]:
train20k['ori'] = train20k['ori'].apply(lambda x: x.replace('toxic: ', '').strip())
train20k['gen'] = train20k['gen'].apply(lambda x: x[:x.index('</s>')].replace('normal ', ''.strip()) \
                                   if '</s>' in x else x.replace('normal ', '').strip())

In [15]:
train['chr_count'] = train['gen'].apply(lambda x: len(x))
dev['chr_count'] = dev['gen'].apply(lambda x: len(x))
test['chr_count'] = test['gen'].apply(lambda x: len(x))

In [7]:
train20k['chr_count'] = train20k['gen'].apply(lambda x: len(x))

In [18]:
train_fil = train[train['chr_count'] > 10]
dev_fil = dev[dev['chr_count'] > 10]
test_fil = test[test['chr_count'] > 10]

In [8]:
train20k_fil = train20k[train20k['chr_count'] > 10]
train20k_fil.head()

,generated,ori,gen,chr_count
0,paraphrase: toxic: mccain was too stupid to p...,mccain was too stupid to pick palin.,McCain was too stunned to pick Palin.,39
1,paraphrase: toxic: this is the dumbest argume...,this is the dumbest argument ive ever heard ag...,This is the dumbest argument I've ever heard ...,67
2,paraphrase: toxic: that stupid movie with nic...,that stupid movie with nicholas cage really de...,Nicolas Cage's movie really demeaned their co...,58
3,"paraphrase: toxic: wankers, the lot of them. ...","wankers, the lot of them.","Wankers, the lot of them.",27
4,paraphrase: toxic: you lefties are such sheep...,you lefties are such sheepletons.,"You lefty are so sheep, to which you belong.",46


In [19]:
train_fil.head()

,generated,ori,gen,chr_count
0,paraphrase: toxic: mccain was too stupid to p...,mccain was too stupid to pick palin.,McCain was too clever to pick Palin.,36
1,paraphrase: toxic: this is the dumbest argume...,this is the dumbest argument ive ever heard ag...,This is the strangest argument against gun con...,51
2,paraphrase: toxic: that stupid movie with nic...,that stupid movie with nicholas cage really de...,Nicolas Cage really demeaned their contribution.,48
3,"paraphrase: toxic: wankers, the lot of them. ...","wankers, the lot of them.","Drinkers, the lot of them.",26
4,paraphrase: toxic: you lefties are such sheep...,you lefties are such sheepletons.,You left-handed people are so very sheep.,41


In [20]:
train.info(), train_fil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   generated  12000 non-null  object
 1   ori        12000 non-null  object
 2   gen        12000 non-null  object
 3   chr_count  12000 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 375.1+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11973 entries, 0 to 11999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   generated  11973 non-null  object
 1   ori        11973 non-null  object
 2   gen        11973 non-null  object
 3   chr_count  11973 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 467.7+ KB


(None, None)

In [10]:
train20k_fil.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19965 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   generated  19965 non-null  object
 1   ori        19965 non-null  object
 2   gen        19965 non-null  object
 3   chr_count  19965 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 779.9+ KB


In [12]:
train20k_fil[['generated', 'ori', 'gen']].to_csv('../../data/paraphrase/output/train_gen_pair_20k_.txt',
                                                index=False, header=True, sep='\t')

##### Preparation

In [23]:
toxics = DatasetDict({
    'train': Dataset.from_pandas(train_fil[['gen']]),
    'validation': Dataset.from_pandas(dev_fil[['gen']]),
    'test': Dataset.from_pandas(test_fil[['gen']])
})

try:
    toxics = toxics.remove_columns(['__index_level_0__'])
except:
    pass

toxics

DatasetDict({
    train: Dataset({
        features: ['gen'],
        num_rows: 11973
    })
    validation: Dataset({
        features: ['gen'],
        num_rows: 6150
    })
    test: Dataset({
        features: ['gen'],
        num_rows: 9569
    })
})

In [24]:
# labels
id2label = {0: 'Normal', 1: 'Toxic'}
label2id = {'Normal': 0, 'Toxic': 1}

In [25]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CKPT)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_CKPT,
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

model = model.to(DEVICE)

sta_model = AutoModelForSequenceClassification.from_pretrained(S_ROBERTA,
                                                               num_labels=2,
                                                               id2label=id2label,
                                                               label2id=label2id)
sta_model = sta_model.to(DEVICE2)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [26]:
# model = load_checkpoint('../model/ft-robertoxic-classifier.pth', model=model)
model.load_state_dict(torch.load('../../classifier/model/ft-robertoxic-classifier.pth'))

<All keys matched successfully>

In [27]:
def tokenize_data(batch):
    try:
        comment = batch['gen.2']
    except:
        comment = batch['gen']
    
    tokenized = tokenizer(
        comment,
        truncation=True,
        max_length=128,
        padding='max_length'
    )

    return tokenized

In [28]:
toxics_tokenized = toxics.map(tokenize_data, batched=True)
toxics_tokenized

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['gen', 'input_ids', 'attention_mask'],
        num_rows: 11973
    })
    validation: Dataset({
        features: ['gen', 'input_ids', 'attention_mask'],
        num_rows: 6150
    })
    test: Dataset({
        features: ['gen', 'input_ids', 'attention_mask'],
        num_rows: 9569
    })
})

In [29]:
toxics_tokenized.set_format('torch')

In [30]:
trainloader = DataLoader(toxics_tokenized['train'], shuffle=False, batch_size=BATCH_SIZE)
devloader = DataLoader(toxics_tokenized['validation'], shuffle=False, batch_size=BATCH_SIZE)
testloader = DataLoader(toxics_tokenized['test'], shuffle=False, batch_size=BATCH_SIZE)

##### Inference

In [31]:
trainpreds = []
train_tox_proba = []
train_norm_proba = []

stapreds = []
sta_tox_proba = []
sta_norm_proba = []

for batch in tqdm(trainloader):
    # self trained model
    batch = {k: v.to(DEVICE) for k, v in batch.items() if k in ['input_ids', 'attention_mask']}
    out = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        
    logits = out.logits.squeeze().detach().cpu()
    probs = torch.sigmoid(logits)    
    preds = torch.argmax(probs, dim=1).numpy()
    
    train_norm_proba.extend(probs[:, 0].tolist())
    train_tox_proba.extend(probs[:, 1].tolist())
    trainpreds.extend(preds.tolist())
    
    # sta model
    batch = {k: v.to(DEVICE2) for k, v in batch.items() if k in ['input_ids', 'attention_mask']}
    out2 = sta_model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])

    logits = out2.logits.squeeze().detach().cpu()
    probs = torch.sigmoid(logits)    
    preds = torch.argmax(probs, dim=1).numpy()
    
    sta_norm_proba.extend(probs[:, 0].tolist())
    sta_tox_proba.extend(probs[:, 1].tolist())
    stapreds.extend(preds.tolist())

  0%|          | 0/375 [00:00<?, ?it/s]

In [32]:
devpreds = []
dev_tox_proba = []
dev_norm_proba = []

dev_stapreds = []
dev_sta_tox_proba = []
dev_sta_norm_proba = []

for batch in tqdm(devloader):
    # self trained model
    batch = {k: v.to(DEVICE) for k, v in batch.items() if k in ['input_ids', 'attention_mask']}
    out = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        
    logits = out.logits.squeeze().detach().cpu()
    probs = torch.sigmoid(logits)    
    preds = torch.argmax(probs, dim=1).numpy()
    
    dev_norm_proba.extend(probs[:, 0].tolist())
    dev_tox_proba.extend(probs[:, 1].tolist())
    devpreds.extend(preds.tolist())
    
    # sta model
    batch = {k: v.to(DEVICE2) for k, v in batch.items() if k in ['input_ids', 'attention_mask']}
    out2 = sta_model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])

    logits = out2.logits.squeeze().detach().cpu()
    probs = torch.sigmoid(logits)    
    preds = torch.argmax(probs, dim=1).numpy()
    
    dev_sta_norm_proba.extend(probs[:, 0].tolist())
    dev_sta_tox_proba.extend(probs[:, 1].tolist())
    dev_stapreds.extend(preds.tolist())

  0%|          | 0/193 [00:00<?, ?it/s]

In [45]:
testpreds = []
test_tox_proba = []
test_norm_proba = []

test_stapreds = []
test_sta_tox_proba = []
test_sta_norm_proba = []

for batch in tqdm(testloader):
    # self trained model
    batch = {k: v.to(DEVICE) for k, v in batch.items() if k in ['input_ids', 'attention_mask']}
    out = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        
    logits = out.logits.squeeze().detach().cpu()
    
    try:
        probs = torch.sigmoid(logits)
        preds = torch.argmax(probs, dim=1).numpy()
    except:
        probs = probs.unsqueeze(0)
        preds = torch.argmax(probs, dim=1).numpy()
    
    test_norm_proba.extend(probs[:, 0].tolist())
    test_tox_proba.extend(probs[:, 1].tolist())
    testpreds.extend(preds.tolist())
    
    # sta model
    batch = {k: v.to(DEVICE2) for k, v in batch.items() if k in ['input_ids', 'attention_mask']}
    out2 = sta_model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])

    logits = out2.logits.squeeze().detach().cpu()
    probs = torch.sigmoid(logits)
    try:
        probs = torch.sigmoid(logits)
        preds = torch.argmax(probs, dim=1).numpy()
    except:
        probs = probs.unsqueeze(0)
        preds = torch.argmax(probs, dim=1).numpy()
    
    test_sta_norm_proba.extend(probs[:, 0].tolist())
    test_sta_tox_proba.extend(probs[:, 1].tolist())
    test_stapreds.extend(preds.tolist())

  0%|          | 0/300 [00:00<?, ?it/s]

In [49]:
train_fil['gen_preds'] = trainpreds
train_fil['norm_proba'] = train_norm_proba
train_fil['tox_proba'] = train_tox_proba

dev_fil['gen_preds'] = devpreds
dev_fil['norm_proba'] = dev_norm_proba
dev_fil['tox_proba'] = dev_tox_proba

test_fil['gen_preds'] = testpreds
test_fil['norm_proba'] = test_norm_proba
test_fil['tox_proba'] = test_tox_proba


train_fil['sta_gen_preds'] = stapreds
train_fil['sta_norm_proba'] = sta_norm_proba
train_fil['sta_tox_proba'] = sta_tox_proba

dev_fil['sta_gen_preds'] = dev_stapreds
dev_fil['sta_norm_proba'] = dev_sta_norm_proba
dev_fil['sta_tox_proba'] = dev_sta_tox_proba

test_fil['sta_gen_preds'] = test_stapreds
test_fil['sta_norm_proba'] = test_sta_norm_proba
test_fil['sta_tox_proba'] = test_sta_tox_proba

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [50]:
train_fil.head()

,generated,ori,gen,chr_count,gen_preds,norm_proba,tox_proba,sta_gen_preds,sta_norm_proba,sta_tox_proba
0,paraphrase: toxic: mccain was too stupid to p...,mccain was too stupid to pick palin.,McCain was too clever to pick Palin.,36,0,0.982992,0.031515,0,0.986560,0.013621
1,paraphrase: toxic: this is the dumbest argume...,this is the dumbest argument ive ever heard ag...,This is the strangest argument against gun con...,51,0,0.994452,0.008538,0,0.988612,0.009887
2,paraphrase: toxic: that stupid movie with nic...,that stupid movie with nicholas cage really de...,Nicolas Cage really demeaned their contribution.,48,0,0.993742,0.009819,0,0.992538,0.006079
3,"paraphrase: toxic: wankers, the lot of them. ...","wankers, the lot of them.","Drinkers, the lot of them.",26,0,0.964932,0.062585,0,0.986371,0.013188
4,paraphrase: toxic: you lefties are such sheep...,you lefties are such sheepletons.,You left-handed people are so very sheep.,41,1,0.305110,0.770181,0,0.744940,0.314475


In [67]:
train_fil[train_fil['sta_gen_preds'] == 0][['ori', 'gen']].to_csv('../../data/parallel/train_gen_pair.txt', sep='\t', index=False, header=True)
dev_fil[dev_fil['sta_gen_preds'] == 0][['ori', 'gen']].to_csv('../../data/parallel/valid_gen_pair.txt', sep='\t', index=False, header=True)
test_fil[test_fil['sta_gen_preds'] == 0][['ori', 'gen']].to_csv('../../data/parallel/test_gen_pair.txt', sep='\t', index=False, header=True)

##### Fluency

In [70]:
CKPT = 'cointegrated/roberta-large-cola-krishna2020'
fl_tokenizer = AutoTokenizer.from_pretrained(CKPT)
fl_model = AutoModelForSequenceClassification.from_pretrained(CKPT).to(DEVICE)

In [ ]:
fl_model

##### Calculate Similarity

In [21]:
def cosine_similarity(x, y):
    return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))

In [27]:
tfidf = TfidfVectorizer(vocabulary=tokenizer.vocab)
train_ori_vec = tfidf.fit_transform(train['ori']).toarray()
train_gen_vec = tfidf.fit_transform(train['gen.2']).toarray()

dev_ori_vec = tfidf.fit_transform(dev['ori']).toarray()
dev_gen_vec = tfidf.fit_transform(dev['gen.2']).toarray()

dev_fft_ori_vec = tfidf.fit_transform(dev_fft['ori']).toarray()
dev_fft_gen_vec = tfidf.fit_transform(dev_fft['gen']).toarray()

test_ori_vec = tfidf.fit_transform(test['ori']).toarray()
test_gen_vec = tfidf.fit_transform(test['gen.2']).toarray()

In [28]:
train_sim = []
dev_sim = []
dev_fft_sim = []
test_sim = []

for gen, ori in tqdm(zip(train_gen_vec, train_ori_vec)):
    train_sim.append(cosine_similarity(gen, ori))
    
for gen, ori in tqdm(zip(dev_gen_vec, dev_ori_vec)):
    dev_sim.append(cosine_similarity(gen, ori))

for gen, ori in tqdm(zip(dev_fft_gen_vec, dev_fft_ori_vec)):
    dev_fft_sim.append(cosine_similarity(gen, ori))
    
for gen, ori in tqdm(zip(test_gen_vec, test_ori_vec)):
    test_sim.append(cosine_similarity(gen, ori))

0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [29]:
train['similarity'] = train_sim
dev['similarity'] = dev_sim
dev_fft['similarity'] = dev_fft_sim
test['similarity'] = test_sim

In [30]:
train.similarity.fillna(0, inplace=True)
dev.similarity.fillna(0, inplace=True)
dev_fft.similarity.fillna(0, inplace=True)
test.similarity.fillna(0, inplace=True)

In [31]:
len(train[(train['similarity'] > .02) & (train['gen_preds'] == 0)])

5813

#### Semantic similarity

In [32]:
from sentence_transformers import SentenceTransformer, util

stmodel = SentenceTransformer('all-MiniLM-L6-v2')
stmodel = stmodel.to(DEVICE)

In [35]:
class SemanticDataset(Dataset):
    def __init__(self, df=None, path=None, base_dir='../data/paraphrase/'):
        
        self.data_list = []
        
        self.path = os.path.join(base_dir, path) if path is not None else None
        
        data = pd.read_csv(self.path, sep='\t') if df is None else df
        for row in tqdm(data.iterrows()):
            self.data_list.append({
                'ori': row[1].ori,
                'gen': row[1]['gen.2']
            })
            
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, item):
        return self.data_list[item]

In [36]:
trainbt_dataset = SemanticDataset(train)
devbt_dataset = SemanticDataset(dev)
testbt_dataset = SemanticDataset(test)

trainbt_loader = DataLoader(trainbt_dataset, batch_size=BATCH_SIZE, shuffle=False)
devbt_loader = DataLoader(devbt_dataset, batch_size=BATCH_SIZE, shuffle=False)
testbt_loader = DataLoader(testbt_dataset, batch_size=BATCH_SIZE, shuffle=False)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [38]:
train_stsim = []

for batch in tqdm(trainbt_loader):
    source = batch['ori']
    backtrans = batch['gen']
    src_embed = stmodel.encode(source, convert_to_tensor=True)
    bts_embed = stmodel.encode(backtrans, convert_to_tensor=True)
    scores = util.cos_sim(src_embed, bts_embed)
    
    for i in range(len(source)):
        train_stsim.append(scores[i][i].item())
        
    torch.cuda.empty_cache()
    del src_embed
    del bts_embed

  0%|          | 0/1471 [00:00<?, ?it/s]

In [39]:
dev_stsim = []

for batch in tqdm(devbt_loader):
    source = batch['ori']
    backtrans = batch['gen']
    src_embed = stmodel.encode(source, convert_to_tensor=True)
    bts_embed = stmodel.encode(backtrans, convert_to_tensor=True)
    scores = util.cos_sim(src_embed, bts_embed)
    
    for i in range(len(source)):
        dev_stsim.append(scores[i][i].item())
        
    torch.cuda.empty_cache()
    del src_embed
    del bts_embed

  0%|          | 0/72 [00:00<?, ?it/s]

In [40]:
test_stsim = []

for batch in tqdm(testbt_loader):
    source = batch['ori']
    backtrans = batch['gen']
    src_embed = stmodel.encode(source, convert_to_tensor=True)
    bts_embed = stmodel.encode(backtrans, convert_to_tensor=True)
    scores = util.cos_sim(src_embed, bts_embed)
    
    for i in range(len(source)):
        test_stsim.append(scores[i][i].item())
        
    torch.cuda.empty_cache()
    del src_embed
    del bts_embed

  0%|          | 0/112 [00:00<?, ?it/s]

In [42]:
train['sem_similarity'] = train_stsim
dev['sem_similarity'] = dev_stsim
test['sem_similarity'] = test_stsim

In [64]:
len(train[(train['similarity'] > .02) & (train['gen_preds'] == 0)])

5813

In [7]:
len(train[(train['chr_count'] > 1) & (train['gen_preds'] == 0) & (train['similarity'] > 0.2)])

3869

In [14]:
len(train[train['gen.2'] == ' ']), len(train[train['gen'] == ' '])

(30140, 28173)

##### save to disk

In [65]:
train.to_csv('../data/paraphrase/train_gen_pair.txt', sep='\t', index=False, header=True)
dev.to_csv('../data/paraphrase/valid_gen_pair.txt', sep='\t', index=False, header=True)
dev_fft.to_csv('../data/paraphrase/valid_fft_gen_pair.txt', sep='\t', index=False, header=True)
test.to_csv('../data/paraphrase/test_gen_pair.txt', sep='\t', index=False, header=True)

##### Make some examples

In [9]:
selected = train[(train['chr_count'] > 1) & (train['sem_similarity'] > .02) & (train['gen_preds'] == 0)].head()

for row in selected.iterrows():
    print(f'original: {row[1]["ori"]}')
    print(f'generated: {row[1]["gen.2"]}')
    print(f'toxic probability: {row[1]["tox_proba"]:.4f}')
    print('*' * 50)

original: you just made a fool of yourself. 
generated:  I don't know why you're so angry. 
toxic probability: 0.0101
**************************************************
original: youre a fucking boot as well. 
generated:  A boot is also pretty good. 
toxic probability: 0.0127
**************************************************
original: these bash pieces have officially become pathetic. 
generated:  The other half of the Shihan is funny. 
toxic probability: 0.0109
**************************************************
original: otherwise, your comment is hypocritical. 
generated:  Other things are less so. 
toxic probability: 0.0081
**************************************************
original: the bc green voter is an immoral puerile malcontent that takes no responsibly for the massive destruction their foolish vote might engender. 
generated:  On the other hand, it's a very good voter who took a good responsibility for the massive destruction of their vote might cause. 
toxic probability: 0

In [95]:
selected = dev_fft[(dev_fft['similarity'] > .02)].head()

for row in selected.iterrows():
    print(f'original: {row[1]["ori"]}')
    print(f'generated: {row[1]["gen"]}')
    print(f'toxic probability: {row[1]["tox_proba"]:.4f}')
    print('*' * 50)

original:  i think youre just pathetic whiner. 
generated:  ive just got a little bit of an asshole. 
toxic probability: 0.9661
**************************************************
original:  ah, there is nothing like a swift kick in the groin from the representativesk pfeffer. 
generated:  , there's nothing like a violent kick in the groin from the representatives of the pimper. 
toxic probability: 0.9266
**************************************************
original:  gays should stop deluding themselves into thinking that the average canadian really cares about gay rights. 
generated:  should stop wasting their lives fooling themselves that the average Canadian really cares about gays. 
toxic probability: 0.8934
**************************************************
original:  facts are not kind to white bigots and the lies they tell themselves about themselves. 
generated:  ices are not kind to the white-hatted racists and lies they tell themselves. 
toxic probability: 0.8975
**************

In [97]:
display(len(train[(train['similarity'] > .02) & (train['gen_preds'] == 0)]))
display(len(dev_fft[(dev_fft['similarity'] > .02) & (dev_fft['gen_preds'] == 0)]))

5949

176